In [12]:
#standard(stats)
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
#shooting(shooting)
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled", "sca_defense", "gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled", "gca_defense"]
#defensive actions(defense)
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
#goalkeeping(keepers)
keepers = ["player","nationality","position","squad","age","birth_year","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
#advance goalkeeping(keepersadv)
keepersadv2 = ["minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]

In [13]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv

# The following functions extract the data into a dataframe

In [47]:
def get_tables(url):
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    team_table = all_tables[0]
    player_table = all_tables[1]
    return player_table, team_table

def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player

def frame_for_category(category,top,end,features):
    url = (top + category + end)
    player_table, team_table = get_tables(url)
    df_player = get_frame(features, player_table)
    return df_player

# This function returns a dataframe with all the statistics of the field players

In [48]:

def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

# This function returns a dataframe with all the statistics of the gatekeepers

In [49]:

def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df3 = frame_for_category('passing_types',top,end,passing_types2)
    df = pd.concat([df1, df2, df3], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

# All the tables of the seasons from 2018 to 2021 are concatenated

In [50]:

df_2018 = get_outfield_data('https://fbref.com/en/comps/Big5/2017-2018/','/players/2017-2018-Big-5-European-Leagues-Stats')
df_2018["player"] = df_2018["player"] + ', 2017-18'
df_2019 = get_outfield_data('https://fbref.com/en/comps/Big5/2018-2019/','/players/2018-2019-Big-5-European-Leagues-Stats')
df_2019["player"] = df_2019["player"] + ', 2018-19'
df_2020 = get_outfield_data('https://fbref.com/en/comps/Big5/2019-2020/','/players/2019-2020-Big-5-European-Leagues-Stats')
df_2020["player"] = df_2020["player"] + ', 2019-20'
df_2021 = get_outfield_data('https://fbref.com/en/comps/Big5/2020-2021/','/players/2020-2021-Big-5-European-Leagues-Stats')
df_2021["player"] = df_2021["player"] + ', 2020-21'
df = pd.concat([df_2018, df_2019, df_2020, df_2021])

df.head()

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,"Patrick van Aanholt, 2017-18",nl NED,DF,Crystal Palace,26,1990,28.0,25.0,2184.0,5.0,...,18.0,10.0,3.0,0.0,0.0,0.0,213.0,7.0,10.0,41.2
1,"Rolando Aarons, 2017-18",eng ENG,"FW,MF",Newcastle Utd,21,1995,4.0,1.0,139.0,0.0,...,8.0,11.0,0.0,0.0,0.0,0.0,22.0,7.0,15.0,31.8
2,"Rolando Aarons, 2017-18",eng ENG,"MF,FW",Hellas Verona,21,1995,11.0,6.0,517.0,0.0,...,1.0,3.0,1.0,0.0,0.0,0.0,13.0,2.0,3.0,40.0
3,"Ignazio Abate, 2017-18",it ITA,DF,Milan,30,1986,17.0,11.0,1057.0,1.0,...,11.0,8.0,3.0,0.0,0.0,0.0,106.0,5.0,4.0,55.6
4,"Aymen Abdennour, 2017-18",tn TUN,DF,Marseille,27,1989,8.0,6.0,500.0,0.0,...,7.0,6.0,0.0,0.0,0.0,0.0,57.0,3.0,0.0,100.0


In [90]:
daat = pd.read_csv('fbref_players.csv')
df_2014 = pd.read_csv('Players_2014.csv')
df_2015 = pd.read_csv('Players_2015.csv')
df_2016 = pd.read_csv('Players_2016.csv')
df_2017 = pd.read_csv('Players_2017.csv')


In [93]:
df = pd.concat([df_2014,df_2015,df_2016,df_2017, df_2018, df_2019, df_2020, df_2021])

df.head(5)

,Unnamed: 0,games_starts_gk,losses,minutes_per_game_gk,save_perc,nationality,player,position,age,clean_sheets_perc,wins,goals_against_per90,games_subs_gk,goals_against,shots_on_target_against,games_gk,clean_sheets,minutes_gk,draws,squad,pens_made,goals_assists_per90,fouls,goals,goals_assists_pens_per90,goals_per90,minutes_per_game_pl,games_starts_pl,games_subs_pl,cards_per90,shots_on_target_per90,goals_pens_per90,games_pl,cards_yellow,cards_red,assists,minutes_pl,fouls_per90,shots_on_target,pens_att,season,birth_year,games,games_starts,minutes,assists_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,minutes_90s,shots_total,shots_free_kicks,shots_on_target_pct,shots_total_per90,goals_per_shot,goals_per_shot_on_target,npxg_per_shot,xg_net,npxg_net,passes_completed,passes,passes_pct,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,passes_medium,passes_pct_medium,passes_completed_long,passes_long,passes_pct_long,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes,passes_live,passes_dead,passes_free_kicks,through_balls,passes_pressure,passes_switches,crosses,corner_kicks,corner_kicks_in,corner_kicks_out,corner_kicks_straight,passes_ground,passes_low,passes_high,passes_left_foot,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_offsides,passes_oob,passes_intercepted,passes_blocked,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_defense,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_defense,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,dribble_tackles,dribbles_vs,dribble_tackles_pct,dribbled_past,pressures,pressure_regains,pressure_regain_pct,pressures_def_3rd,pressures_mid_3rd,pressures_att_3rd,blocks,blocked_shots,blocked_shots_saves,blocked_passes,interceptions,clearances,errors,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,pass_targets,passes_received,passes_received_pct,miscontrols,dispossessed,cards_yellow_red,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,0.0,38.0,19.0,90.0,0.704,eng,Tom Heaton,GK,28,26.3,7.0,1.39,0.0,53.0,179.0,38.0,10.0,3420.0,12.0,Burnley,0.0,0.0,0.0,0.0,0.0,0.0,90.0,38.0,0.0,0.03,0.0,0.0,38.0,1.0,0.0,0.0,3420.0,0.0,0.0,0.0,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,38.0,15.0,89.0,0.725,es,Adrián,GK,27,21.1,12.0,1.25,0.0,47.0,171.0,38.0,8.0,3392.0,10.0,West Ham United,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,37.0,8

In [ ]:
df.to_csv('fbref_full_players.csv')
df

In [95]:
pd.DataFrame(df.columns)

,0
0,Unnamed: 0
1,games_starts_gk
2,losses
3,minutes_per_game_gk
4,save_perc
5,nationality
6,player
7,position
8,age
9,clean_sheets_perc


In [75]:
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_columns',None)
# pd.set_option('display.width',None)

In [76]:
pd.DataFrame(df.columns)

,0
0,player
1,nationality
2,position
3,squad
4,age
5,birth_year
6,games
7,games_starts
8,minutes
9,goals


In [69]:
df_2015 = get_outfield_data('https://fbref.com/en/comps/Big5/2014-2015/','/players/2014-2015-Big-5-European-Leagues-Stats')


AttributeError: 'NoneType' object has no attribute 'text'

In [51]:
df.to_csv('fbref_players.csv')
df

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,"Patrick van Aanholt, 2017-18",nl NED,DF,Crystal Palace,26,1990,28.0,25.0,2184.0,5.0,...,18.0,10.0,3.0,0.0,0.0,0.0,213.0,7.0,10.0,41.2
1,"Rolando Aarons, 2017-18",eng ENG,"FW,MF",Newcastle Utd,21,1995,4.0,1.0,139.0,0.0,...,8.0,11.0,0.0,0.0,0.0,0.0,22.0,7.0,15.0,31.8
2,"Rolando Aarons, 2017-18",eng ENG,"MF,FW",Hellas Verona,21,1995,11.0,6.0,517.0,0.0,...,1.0,3.0,1.0,0.0,0.0,0.0,13.0,2.0,3.0,40.0
3,"Ignazio Abate, 2017-18",it ITA,DF,Milan,30,1986,17.0,11.0,1057.0,1.0,...,11.0,8.0,3.0,0.0,0.0,0.0,106.0,5.0,4.0,55.6
4,"Aymen Abdennour, 2017-18",tn TUN,DF,Marseille,27,1989,8.0,6.0,500.0,0.0,...,7.0,6.0,0.0,0.0,0.0,0.0,57.0,3.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,"Steven Zuber, 2020-21",ch SUI,"DF,MF",Eint Frankfurt,28,1991,20.0,6.0,585.0,0.0,...,14.0,11.0,1.0,0.0,0.0,0.0,65.0,1.0,10.0,9.1
2818,"Martín Zubimendi, 2020-21",es ESP,MF,Real Sociedad,21,1999,31.0,17.0,1882.0,0.0,...,37.0,17.0,0.0,0.0,0.0,0.0,187.0,52.0,28.0,65.0
2819,"Martin Ødegaard, 2020-21",no NOR,MF,Arsenal,21,1998,14.0,9.0,866.0,1.0,...,1.0,9.0,1.0,0.0,0.0,0.0,56.0,1.0,14.0,6.7
2820,"Martin Ødegaard, 2020-21",no NOR,"MF,FW",Real Madrid,21,1998,7.0,3.0,234.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,15.0,2.0,3.0,40.0


# A list is extracted with the links of all the players

In [52]:
player_table_2018 = get_tables('https://fbref.com/en/comps/Big5/2017-2018/stats/players/2017-2018-Big-5-European-Leagues-Stats')[0]
rows_player_2018 = player_table_2018.find_all('tr')
rows_player_2018
link_jugador_2018 = list()
for row in rows_player_2018:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_jugador_2018.append(jugador.attrs['href'])
                
player_table_2019 = get_tables('https://fbref.com/en/comps/Big5/2018-2019/stats/players/2018-2019-Big-5-European-Leagues-Stats')[0]
rows_player_2019 = player_table_2019.find_all('tr')
rows_player_2019
link_jugador_2019 = list()
for row in rows_player_2019:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_jugador_2019.append(jugador.attrs['href'])
                
player_table_2020 = get_tables('https://fbref.com/en/comps/Big5/2019-2020/stats/players/2019-2020-Big-5-European-Leagues-Stats')[0]
rows_player_2020 = player_table_2020.find_all('tr')
rows_player_2020
link_jugador_2020 = list()
for row in rows_player_2020:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_jugador_2020.append(jugador.attrs['href'])
                
player_table_2021 = get_tables('https://fbref.com/en/comps/Big5/2020-2021/stats/players/2020-2021-Big-5-European-Leagues-Stats')[0]
rows_player_2021 = player_table_2021.find_all('tr')
rows_player_2021
link_jugador_2021 = list()
for row in rows_player_2021:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_jugador_2021.append(jugador.attrs['href'])
                
link_jugador = link_jugador_2018+link_jugador_2019+link_jugador_2020+link_jugador_2021

# You get a link with the photos of all the players

In [53]:
image_jugador = link_jugador.copy()
for i in range(0, len(link_jugador)):
    image_jugador[i] = "https://fbref.com/req/202005121/images/headshots/" + link_jugador[i][12:20] + "_2018.jpg"

#  A new column is added to the dataframe: the links to the images of the players

In [54]:
df["Image jugador"] = image_jugador

# This dataframe is finalized by eliminating the possible gatekeepers who have sneaked in

In [55]:
outfield = df[(df.position != "GK") & (df.position != "GK,MF")]

In [ ]:
df_keeper_2014 = pd.read_csv('Players_2014')
df_keeper_2015 = pd.read_csv('Players_2015')
df_keeper_2016 = pd.read_csv('Players_2016')
df_keeper_2017 = pd.read_csv('Players_')

# The same process is carried out with the goalkeepers

In [56]:

df_keeper_2018 = get_keeper_data('https://fbref.com/en/comps/Big5/2017-2018/','/players/2017-2018-Big-5-European-Leagues-Stats')
df_keeper_2018["player"] = df_keeper_2018["player"] + ', 2017-18'
df_keeper_2019 = get_keeper_data('https://fbref.com/en/comps/Big5/2018-2019/','/players/2018-2019-Big-5-European-Leagues-Stats')
df_keeper_2019["player"] = df_keeper_2019["player"] + ', 2018-19'
df_keeper_2020 = get_keeper_data('https://fbref.com/en/comps/Big5/2019-2020/','/players/2019-2020-Big-5-European-Leagues-Stats')
df_keeper_2020["player"] = df_keeper_2020["player"] + ', 2019-20'
df_keeper_2021 = get_keeper_data('https://fbref.com/en/comps/Big5/2020-2021/','/players/2020-2021-Big-5-European-Leagues-Stats')
df_keeper_2021["player"] = df_keeper_2021["player"] + ', 2020-21'
df_keeper = pd.concat([df_keeper_2018, df_keeper_2019, df_keeper_2020, df_keeper_2021, df_keeper_2014, df_keeper_2015, df_keeper_2016])
df_keeper.head()

,player,nationality,position,squad,age,birth_year,games_gk,games_starts_gk,minutes_gk,goals_against_gk,...,passes_left_foot,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_completed,passes_offsides,passes_oob,passes_intercepted,passes_blocked
0,"Antonio Adán, 2017-18",es ESP,GK,Betis,30,1987,30.0,30.0,2700.0,53.0,...,731.0,163.0,23.0,233.0,4.0,886.0,3.0,18.0,35.0,32.0
1,"René Adler, 2017-18",de GER,GK,Mainz 05,32,1985,14.0,14.0,1260.0,18.0,...,79.0,26.0,2.0,2.0,0.0,81.0,0.0,9.0,1.0,7.0
2,"Adrián, 2017-18",es ESP,GK,West Ham,30,1987,19.0,19.0,1710.0,29.0,...,28.0,10.0,3.0,0.0,0.0,30.0,1.0,1.0,2.0,3.0
3,"Alisson, 2017-18",br BRA,GK,Roma,24,1992,37.0,37.0,3330.0,28.0,...,65.0,584.0,16.0,104.0,0.0,625.0,4.0,14.0,9.0,26.0
4,"Sergio Álvarez, 2017-18",es ESP,GK,Celta Vigo,30,1986,17.0,16.0,1479.0,29.0,...,294.0,20.0,17.0,0.0,2.0,304.0,1.0,6.0,2.0,4.0


In [96]:
pd.DataFrame(df_keeper.columns)

,0
0,player
1,nationality
2,position
3,squad
4,age
5,birth_year
6,games_gk
7,games_starts_gk
8,minutes_gk
9,goals_against_gk


In [57]:
df_keeper.to_csv('fbref_goalkeepers.csv')
df_keeper

,player,nationality,position,squad,age,birth_year,games_gk,games_starts_gk,minutes_gk,goals_against_gk,...,passes_left_foot,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_completed,passes_offsides,passes_oob,passes_intercepted,passes_blocked
0,"Antonio Adán, 2017-18",es ESP,GK,Betis,30,1987,30.0,30.0,2700.0,53.0,...,731.0,163.0,23.0,233.0,4.0,886.0,3.0,18.0,35.0,32.0
1,"René Adler, 2017-18",de GER,GK,Mainz 05,32,1985,14.0,14.0,1260.0,18.0,...,79.0,26.0,2.0,2.0,0.0,81.0,0.0,9.0,1.0,7.0
2,"Adrián, 2017-18",es ESP,GK,West Ham,30,1987,19.0,19.0,1710.0,29.0,...,28.0,10.0,3.0,0.0,0.0,30.0,1.0,1.0,2.0,3.0
3,"Alisson, 2017-18",br BRA,GK,Roma,24,1992,37.0,37.0,3330.0,28.0,...,65.0,584.0,16.0,104.0,0.0,625.0,4.0,14.0,9.0,26.0
4,"Sergio Álvarez, 2017-18",es ESP,GK,Celta Vigo,30,1986,17.0,16.0,1479.0,29.0,...,294.0,20.0,17.0,0.0,2.0,304.0,1.0,6.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111.0,195.0,12.0,17.0,1.0,256.0,3.0,7.0,5.0,22.0
2818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,259.0,755.0,50.0,2.0,2.0,982.0,3.0,14.0,14.0,12.0
2819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,484.0,30.0,4.0,1.0,1.0,450.0,0.0,2.0,12.0,9.0
2820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,160.0,9.0,2.0,0.0,2.0,147.0,2.0,1.0,2.0,4.0


In [58]:
df_keeper = df_keeper.dropna()

In [59]:
df_keeper.position = "GK"

/home/bilel13/.pyenv/versions/3.8.6/envs/football_data_scouting/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [60]:
keeper_table_2018 = get_tables('https://fbref.com/en/comps/Big5/2017-2018/keepers/players/2017-2018-Big-5-European-Leagues-Stats')[0]
rows_keeper_2018 = keeper_table_2018.find_all('tr')
rows_keeper_2018
link_portero_2018 = list()
for row in rows_keeper_2018:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_portero_2018.append(jugador.attrs['href'])
                
keeper_table_2019 = get_tables('https://fbref.com/en/comps/Big5/2018-2019/keepers/players/2018-2019-Big-5-European-Leagues-Stats')[0]
rows_keeper_2019 = keeper_table_2019.find_all('tr')
rows_keeper_2019
link_portero_2019 = list()
for row in rows_keeper_2019:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_portero_2019.append(jugador.attrs['href'])
                
keeper_table_2020 = get_tables('https://fbref.com/en/comps/Big5/2019-2020/keepers/players/2019-2020-Big-5-European-Leagues-Stats')[0]
rows_keeper_2020 = keeper_table_2020.find_all('tr')
rows_keeper_2020
link_portero_2020 = list()
for row in rows_keeper_2020:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_portero_2020.append(jugador.attrs['href'])
                
keeper_table_2021 = get_tables('https://fbref.com/en/comps/Big5/2020-2021/keepers/players/2020-2021-Big-5-European-Leagues-Stats')[0]
rows_keeper_2021 = keeper_table_2021.find_all('tr')
rows_keeper_2021
link_portero_2021 = list()
for row in rows_keeper_2021:
    if(row.find('th',{"scope":"row"}) != None):
        cell = row.find("td",{"data-stat":"player"})
        for jugador in cell.find_all('a'):
            if jugador.has_attr('href'):
                link_portero_2021.append(jugador.attrs['href'])
                
link_portero = link_portero_2018+link_portero_2019+link_portero_2020+link_portero_2021

In [61]:
image_portero = link_portero.copy()
for i in range(0, len(link_portero)):
    image_portero[i] = "https://fbref.com/req/202005121/images/headshots/" + link_portero[i][12:20] + "_2018.jpg"

In [62]:
df_keeper["Image jugador"] = image_portero

/tmp/ipykernel_16474/3731399331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keeper["Image jugador"] = image_portero


In [63]:
df_keeper

,player,nationality,position,squad,age,birth_year,games_gk,games_starts_gk,minutes_gk,goals_against_gk,...,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_completed,passes_offsides,passes_oob,passes_intercepted,passes_blocked,Image jugador
0,"Antonio Adán, 2017-18",es ESP,GK,Betis,30,1987,30.0,30.0,2700.0,53.0,...,163.0,23.0,233.0,4.0,886.0,3.0,18.0,35.0,32.0,https://fbref.com/req/202005121/images/headsho...
1,"René Adler, 2017-18",de GER,GK,Mainz 05,32,1985,14.0,14.0,1260.0,18.0,...,26.0,2.0,2.0,0.0,81.0,0.0,9.0,1.0,7.0,https://fbref.com/req/202005121/images/headsho...
2,"Adrián, 2017-18",es ESP,GK,West Ham,30,1987,19.0,19.0,1710.0,29.0,...,10.0,3.0,0.0,0.0,30.0,1.0,1.0,2.0,3.0,https://fbref.com/req/202005121/images/headsho...
3,"Alisson, 2017-18",br BRA,GK,Roma,24,1992,37.0,37.0,3330.0,28.0,...,584.0,16.0,104.0,0.0,625.0,4.0,14.0,9.0,26.0,https://fbref.com/req/202005121/images/headsho...
4,"Sergio Álvarez, 2017-18",es ESP,GK,Celta Vigo,30,1986,17.0,16.0,1479.0,29.0,...,20.0,17.0,0.0,2.0,304.0,1.0,6.0,2.0,4.0,https://fbref.com/req/202005121/images/headsho...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,"Rubén Yáñez, 2020-21",es ESP,GK,Getafe,26,1993,10.0,10.0,900.0,12.0,...,17.0,2.0,0.0,0.0,22.0,0.0,0.0,2.0,0.0,https://fbref.com/req/202005121/images/headsho...
204,"Yoel, 2020-21",es ESP,GK,Eibar,31,1988,4.0,3.0,300.0,3.0,...,354.0,17.0,20.0,2.0,361.0,0.0,8.0,19.0,13.0,https://fbref.com/req/202005121/images/headsho...
205,"Robin Zentner, 2020-21",de GER,GK,Mainz 05,25,1994,31.0,31.0,2790.0,46.0,...,186.0,29.0,0.0,11.0,195.0,2.0,5.0,11.0,14.0,https://fbref.com/req/202005121/images/headsho...
206,"Ron-Robert Zieler, 2020-21",de GER,GK,Köln,31,1989,1.0,0.0,51.0,1.0,...,516.0,31.0,2.0,5.0,468.0,1.0,11.0,15.0,30.0,https://fbref.com/req/202005121/images/headsho...


In [66]:
dat2017 = pd.read_csv('Players_2014.csv')
dat2017

,Unnamed: 0,games_starts_gk,losses,minutes_per_game_gk,save_perc,nationality,player,position,age,clean_sheets_perc,...,goals_pens_per90,games_pl,cards_yellow,cards_red,assists,minutes_pl,fouls_per90,shots_on_target,pens_att,season
0,0,38.0,19.0,90.0,0.704,eng,Tom Heaton,GK,28,26.3,...,0.0,38.0,1.0,0.0,0.0,3420.0,0.0,0.0,0.0,2014
1,1,38.0,15.0,89.0,0.725,es,Adrián,GK,27,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014
2,2,37.0,8.0,90.0,0.719,es,David de Gea,GK,23,29.7,...,0.0,37.0,0.0,0.0,0.0,3313.0,0.0,0.0,0.0,2014
3,3,37.0,13.0,89.0,0.746,pl,Łukasz Fabiański,GK,29,35.1,...,0.0,37.0,0.0,1.0,0.0,3308.0,0.0,0.0,0.0,2014
4,4,36.0,7.0,90.0,0.714,eng,Joe Hart,GK,27,38.9,...,0.0,36.0,1.0,0.0,0.0,3240.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,526,NaN,NaN,NaN,NaN,NaN,Dominic Gape,MF,19,NaN,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2014
527,527,NaN,NaN,NaN,NaN,eng,Ryan Seager,FW,18,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014
528,528,NaN,NaN,NaN,NaN,eng,Tom Thorpe,DF,21,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014
529,529,NaN,NaN,NaN,NaN,eng,Ollie Shenton,MF,16,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014


In [68]:
datfullplayer = pd.read_csv('Full_Player.csv')
datfullplayer.columns

Index(['Unnamed: 0', 'games_starts_gk', 'losses', 'minutes_per_game_gk',
       'save_perc', 'nationality', 'player', 'position', 'age',
       'clean_sheets_perc', 'wins', 'goals_against_per90', 'games_subs_gk',
       'goals_against', 'shots_on_target_against', 'games_gk', 'clean_sheets',
       'minutes_gk', 'draws', 'squad', 'pens_made', 'goals_assists_per90',
       'fouls', 'goals', 'goals_assists_pens_per90', 'goals_per90',
       'minutes_per_game_pl', 'games_starts_pl', 'games_subs_pl',
       'cards_per90', 'shots_on_target_per90', 'goals_pens_per90', 'games_pl',
       'cards_yellow', 'cards_red', 'assists', 'minutes_pl', 'fouls_per90',
       'shots_on_target', 'pens_att', 'season'],
      dtype='object')

In [81]:
X = pd.concat([datfullplayer, df])

In [ ]:
X